# Preprocessing

In [14]:
import pandas as pd
import os

In [3]:
# Paths
IMDB_DATA_PATH = "PATH"
OUTPUT_PATH = "PATH"

# 1. Load IMDb Datasets
print("Loading datasets...")
basics = pd.read_csv(os.path.join(IMDB_DATA_PATH, "title.basics.tsv"), sep='\t', na_values='\\N')
crew = pd.read_csv(os.path.join(IMDB_DATA_PATH, "title.crew.tsv"), sep='\t', na_values='\\N')
principals = pd.read_csv(os.path.join(IMDB_DATA_PATH, "title.principals.tsv"), sep='\t', na_values='\\N')
names = pd.read_csv(os.path.join(IMDB_DATA_PATH, "name.basics.tsv"), sep='\t', na_values='\\N')
ratings = pd.read_csv(os.path.join(IMDB_DATA_PATH, "title.ratings.tsv"), sep='\t', na_values='\\N')

Loading datasets...


C:\Users\Onur Ege\AppData\Local\Temp\ipykernel_2612\17096404.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv(os.path.join(IMDB_DATA_PATH, "title.basics.tsv"), sep='\t', na_values='\\N')


### movies

In [4]:
# 2. Filter only Movies (excluding Episodes)
print("Filtering clean movies...")

# Keep only movies
movies = basics[basics['titleType'] == 'movie']

# Drop movies with titles that indicate episodes
episode_indicators = ['Episode', 'episode', '#', 'Part', 'part']

def is_episode(title):
    return any(indicator in str(title) for indicator in episode_indicators)

movies = movies[~movies['primaryTitle'].apply(is_episode)]

# Drop NA values
movies = movies[['tconst', 'primaryTitle', 'startYear', 'genres']].dropna()

# Merge ratings
movies = movies.merge(ratings, on='tconst', how='left')
movies = movies.dropna(subset=['averageRating', 'numVotes'])

print(f"Number of cleaned movies: {len(movies)}")

# Save Movie nodes
movies_out = movies.rename(columns={
    'tconst': 'id',
    'primaryTitle': 'title',
    'startYear': 'year',
    'averageRating': 'rating',
    'numVotes': 'votes'
})
movies_out['year'] = movies_out['year'].astype(int)
movies_out.to_csv(os.path.join(OUTPUT_PATH, "movies.csv"), index=False)

Filtering clean movies...
Number of cleaned movies: 316159


In [6]:
movies.tail()

,tconst,primaryTitle,startYear,genres,averageRating,numVotes
534162,tt9916362,Coven,2020.0,"Drama,History",6.4,6035.0
534163,tt9916428,The Secret of China,2019.0,"Adventure,History,War",4.7,22.0
534164,tt9916538,Kuambil Lagi Hatiku,2019.0,Drama,8.3,10.0
534167,tt9916706,Dankyavar Danka,2013.0,Comedy,7.7,9.0
534168,tt9916730,6 Gunn,2017.0,Drama,7.0,13.0


### genres

In [9]:
# 3. Extract Genres separately
print("Extracting genres...")
all_genres = set()
movies_out['genres'].str.split(',').apply(all_genres.update)
genres_out = pd.DataFrame({'name': list(all_genres)})
genres_out.to_csv(os.path.join(OUTPUT_PATH, "genres.csv"), index=False)

Extracting genres...


In [10]:
genres_out.head()

,name
0,Adventure
1,Sport
2,Animation
3,Drama
4,Action


### people

In [19]:
# 4. Filter People (Actors, Directors)
print("Filtering actors and directors...")
principals = principals[principals['category'].isin(['actor', 'actress', 'director'])]

# Only keep top 8 actors/actresses per movie + directors
actors = principals[(principals['category'].isin(['actor', 'actress'])) & (principals['ordering'] <= 8)]
directors = principals[principals['category'] == 'director']

Filtering actors and directors...


In [27]:
actors = actors[['nconst', 'tconst']]
actors.columns = ['personId', 'movieId']
actors.to_csv(os.path.join(OUTPUT_PATH, "actors.csv"), index=False)

directors = directors[['nconst', 'tconst']]
directors.columns = ['personId', 'movieId']
directors.to_csv(os.path.join(OUTPUT_PATH, "directors.csv"), index=False)

In [29]:
actors.head()

,personId,movieId
14,nm0443482,tt0000005
15,nm0653042,tt0000005
17,nm0179163,tt0000007
18,nm0183947,tt0000007
24,nm0653028,tt0000008


In [30]:
directors.head()

,personId,movieId
1,nm0005690,tt0000001
4,nm0721526,tt0000002
6,nm0721526,tt0000003
12,nm0721526,tt0000004
19,nm0005690,tt0000007


In [35]:
# 5. Prepare People Nodes
print("Preparing people nodes...")
people_ids = set(actors['personId']).union(directors['personId'])
people = names[names['nconst'].isin(people_ids)][['nconst', 'primaryName', 'birthYear', 'primaryProfession']]
people = people.rename(columns={
    'nconst': 'id',
    'primaryName': 'name'
})
people.to_csv(os.path.join(OUTPUT_PATH, "people.csv"), index=False)

Preparing people nodes...


In [36]:
people.head()

,id,name,birthYear,primaryProfession
0,nm0000001,Fred Astaire,1899.0,"actor,miscellaneous,producer"
1,nm0000002,Lauren Bacall,1924.0,"actress,soundtrack,archive_footage"
2,nm0000003,Brigitte Bardot,1934.0,"actress,music_department,producer"
3,nm0000004,John Belushi,1949.0,"actor,writer,music_department"
4,nm0000005,Ingmar Bergman,1918.0,"writer,director,actor"


# Import to Neo4j

In [12]:
from py2neo import Graph, Node, Relationship

In [13]:
# Connect to Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "PASSWORD"))

In [20]:
# CREATE CONSTRAİNTS
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:Actor) REQUIRE u.id IS UNIQUE;")
graph.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:Director) REQUIRE u.id IS UNIQUE;")

(No data)

In [6]:
# READ CSV 
movies_df = pd.read_csv("C:/Users/Onur Ege/PycharmProjects/GraphDBProject/dataset/processed/movies.csv")
genres_df = pd.read_csv("C:/Users/Onur Ege/PycharmProjects/GraphDBProject/dataset/processed/genres.csv")
people_df = pd.read_csv("C:/Users/Onur Ege/PycharmProjects/GraphDBProject/dataset/processed/people.csv")
actors_df = pd.read_csv("C:/Users/Onur Ege/PycharmProjects/GraphDBProject/dataset/processed/actors.csv")
directors_df = pd.read_csv("C:/Users/Onur Ege/PycharmProjects/GraphDBProject/dataset/processed/directors.csv")

In [15]:
movies_df.shape

(316159, 6)

### insert movies

##### some movies dont have any actor(short movie,audio book), delete them

In [9]:
# Code for inserting movies
batch_size = 500  # size

def insert_movies_part(movies_df_part, graph, part_name="Part"):
    tx = graph.begin()
    
    for index, row in movies_df_part.iterrows():
        # Safely parse genres
        genres = []
        if pd.notna(row.get('genres')):
            if isinstance(row['genres'], str):
                try:
                    genres = eval(row['genres'])  # if genres stored like '["Action", "Comedy"]'
                except:
                    genres = row['genres'].split('|')  # if genres stored like "Action|Comedy"
            else:
                genres = row['genres']

        movie = Node("Movie",
                     id=row['id'],
                     title=row['title'],
                     year=int(row['year']) if not pd.isna(row['year']) else None,
                     rating=float(row['rating']) if not pd.isna(row['rating']) else None,
                     votes=int(row['votes']) if not pd.isna(row['votes']) else None,
                     genres=genres)

        tx.merge(movie, "Movie", "id")
        
        if index % batch_size == 0 and index != 0:
            tx.commit()
            print(f"✅ {part_name} - Committed at index {index}")
            tx = graph.begin()

    tx.commit()
    print(f"✅✅ {part_name} Finished inserting movies with genres!")

In [11]:
# Part 1
#insert_movies_part(movies_df.iloc[0:50000], graph, part_name="Part 1")

# Part 2
#insert_movies_part(movies_df.iloc[50000:100000], graph, part_name="Part 2")

# Part 3
#insert_movies_part(movies_df.iloc[100000:150000], graph, part_name="Part 3")

# Part 4
#insert_movies_part(movies_df.iloc[150000:200000], graph, part_name="Part 4")

# Part 5
#insert_movies_part(movies_df.iloc[200000:250000], graph, part_name="Part 5")

# Part 6
#insert_movies_part(movies_df.iloc[250000:300000], graph, part_name="Part 6")

# Part 7
#insert_movies_part(movies_df.iloc[300000:], graph, part_name="Part 7")

### insert genres

In [10]:
tx = graph.begin()

for index, row in genres_df.iterrows():
    genre = Node("Genre", name=row['name'])
    tx.merge(genre, "Genre", "name")
    
    if index % 500 == 0 and index != 0:
        tx.commit()
        print(f"✅ Inserted {index} genres")
        tx = graph.begin()

tx.commit()
print("✅✅ All genres inserted!")

✅✅ All genres inserted!


C:\Users\Onur Ege\AppData\Local\Temp\ipykernel_22928\3844082110.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


### Insert Person Nodes (Actors and Directors)

In [12]:
def insert_person_part(people_df,graph,part_name="Part"):
    tx = graph.begin()
    
    for index, row in people_df.iterrows():
        professions = str(row['primaryProfession']).split(',')
        person = Node("Person",
                      id=row['id'],
                      name=row['name'],
                      birthYear=int(row['birthYear']) if not pd.isna(row['birthYear']) else None,
                      primaryProfession=professions)
        tx.merge(person, "Person", "id")
        
        if 'actor' in professions or 'actress' in professions:
            person.add_label("Actor")
        if 'director' in professions:
            person.add_label("Director")
            
        graph.push(person)

        if index % 500 == 0 and index != 0:
            tx.commit()
            print(f"✅ Inserted {index} people")
            tx = graph.begin()
    
    tx.commit()
    print("✅✅ All people inserted!")

In [8]:
people_df.shape

(3675633, 4)

In [5]:
people_df.tail()

,id,name,birthYear,primaryProfession
3675628,nm9993700,Sexy Angel,NaN,actress
3675629,nm9993701,Sanjai Kuriakose,NaN,actor
3675630,nm9993703,James Craigmyle,NaN,actor
3675631,nm9993708,Eli Bevins,NaN,"producer,director,writer"
3675632,nm9993709,Lu Bevins,NaN,"producer,writer,director"


In [1]:
#for i in range(1,37):
   # insert_person_part(people_df.iloc[(i-1)*100000:i*100000], graph, part_name='Part ' + str(i))
#insert_person_part(people_df.iloc[3500000:], graph, part_name='Part 37')

### create IN_GENRE relationship

In [22]:
# 3. Define APOC query for IN_GENRE relationships
in_genre_query = """
CALL apoc.periodic.iterate(
  "
  MATCH (m:Movie)
  WHERE NOT (m)-[:IN_GENRE]->(:Genre) AND m.genres IS NOT NULL
  RETURN m
  ",
  "
  UNWIND m.genres AS genreName
  MATCH (g:Genre {name: genreName})
  MERGE (m)-[:IN_GENRE]->(g)
  ",
  {batchSize:500, parallel:false}
)
"""

# 4. Run the APOC batch query
graph.run(in_genre_query)

print("🎯 IN_GENRE relationships creation completed successfully with APOC!")


🎯 IN_GENRE relationships creation completed successfully with APOC!


### create ACTED_IN relationship

In [16]:
actors_df.shape

(32892665, 2)

In [17]:
# Prepare rows
acted_in_row = actors_df[['personId', 'movieId']].to_dict('records')

In [18]:
len(acted_in_row)

32892665

In [27]:
import math

chunk_size = 30000  # Number of rows to send per chunk
total_chunks = math.ceil(len(acted_in_row) / chunk_size)

print(f"🔵 Total chunks to send: {total_chunks}")

for chunk_idx in range(total_chunks):
    chunk_rows = acted_in_row[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]
    
    print(f"🚀 Sending chunk {chunk_idx + 1}/{total_chunks} (rows {len(chunk_rows)})")

    acted_in_query = """
    CALL apoc.periodic.iterate(
      'UNWIND $rows AS row RETURN row',
      '
        WITH row
        MATCH (a:Actor {id: row.personId})
        MATCH (m:Movie {id: row.movieId})
        MERGE (a)-[:ACTED_IN]->(m)
      ',
      {
        batchSize:2000,
        parallel:false,
        params: {rows: $rows}
      }
    )
    """

    graph.run(acted_in_query, parameters={"rows": chunk_rows})

    print(f"✅ Chunk {chunk_idx + 1} finished.")

print("🎯 All ACTED_IN relationships created successfully!")

🔵 Total chunks to send: 1097
🚀 Sending chunk 1/1097 (rows 30000)
✅ Chunk 1 finished.
🚀 Sending chunk 2/1097 (rows 30000)
✅ Chunk 2 finished.
🚀 Sending chunk 3/1097 (rows 30000)
✅ Chunk 3 finished.
🚀 Sending chunk 4/1097 (rows 30000)
✅ Chunk 4 finished.
🚀 Sending chunk 5/1097 (rows 30000)
✅ Chunk 5 finished.
🚀 Sending chunk 6/1097 (rows 30000)
✅ Chunk 6 finished.
🚀 Sending chunk 7/1097 (rows 30000)
✅ Chunk 7 finished.
🚀 Sending chunk 8/1097 (rows 30000)
✅ Chunk 8 finished.
🚀 Sending chunk 9/1097 (rows 30000)
✅ Chunk 9 finished.
🚀 Sending chunk 10/1097 (rows 30000)
✅ Chunk 10 finished.
🚀 Sending chunk 11/1097 (rows 30000)
✅ Chunk 11 finished.
🚀 Sending chunk 12/1097 (rows 30000)
✅ Chunk 12 finished.
🚀 Sending chunk 13/1097 (rows 30000)
✅ Chunk 13 finished.
🚀 Sending chunk 14/1097 (rows 30000)
✅ Chunk 14 finished.
🚀 Sending chunk 15/1097 (rows 30000)
✅ Chunk 15 finished.
🚀 Sending chunk 16/1097 (rows 30000)
✅ Chunk 16 finished.
🚀 Sending chunk 17/1097 (rows 30000)
✅ Chunk 17 finished.
🚀 S

KeyboardInterrupt: 

### create DIRECTED relationship

In [30]:
directors_df.shape

(7936130, 2)

In [33]:
import math

chunk_size = 30000  # Number of rows to send per chunk
total_chunks = math.ceil(len(directors_df) / chunk_size)

print(f"🔵 Total chunks to send: {total_chunks}")

for chunk_idx in range(total_chunks):
    chunk_rows = directors_df[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]
    
    print(f"🚀 Sending chunk {chunk_idx + 1}/{total_chunks} (rows {len(chunk_rows)})")

    directed_query = """
    CALL apoc.periodic.iterate(
      'UNWIND $rows AS row RETURN row',
      '
        WITH row
        MATCH (d:Director {id: row.personId})
        MATCH (m:Movie {id: row.movieId})
        MERGE (d)-[:DIRECTED]->(m)
      ',
      {
        batchSize:2000,
        parallel:false,
        params: {rows: $rows}
      }
    )
    """

    graph.run(directed_query, parameters={"rows": chunk_rows.to_dict('records')})

    print(f"✅ Chunk {chunk_idx + 1} finished.")

print("🎯 All DIRECTED relationships created successfully!")


🔵 Total chunks to send: 265
🚀 Sending chunk 1/265 (rows 30000)
✅ Chunk 1 finished.
🚀 Sending chunk 2/265 (rows 30000)
✅ Chunk 2 finished.
🚀 Sending chunk 3/265 (rows 30000)
✅ Chunk 3 finished.
🚀 Sending chunk 4/265 (rows 30000)
✅ Chunk 4 finished.
🚀 Sending chunk 5/265 (rows 30000)
✅ Chunk 5 finished.
🚀 Sending chunk 6/265 (rows 30000)
✅ Chunk 6 finished.
🚀 Sending chunk 7/265 (rows 30000)
✅ Chunk 7 finished.
🚀 Sending chunk 8/265 (rows 30000)
✅ Chunk 8 finished.
🚀 Sending chunk 9/265 (rows 30000)
✅ Chunk 9 finished.
🚀 Sending chunk 10/265 (rows 30000)
✅ Chunk 10 finished.
🚀 Sending chunk 11/265 (rows 30000)
✅ Chunk 11 finished.
🚀 Sending chunk 12/265 (rows 30000)
✅ Chunk 12 finished.
🚀 Sending chunk 13/265 (rows 30000)
✅ Chunk 13 finished.
🚀 Sending chunk 14/265 (rows 30000)
✅ Chunk 14 finished.
🚀 Sending chunk 15/265 (rows 30000)
✅ Chunk 15 finished.
🚀 Sending chunk 16/265 (rows 30000)
✅ Chunk 16 finished.
🚀 Sending chunk 17/265 (rows 30000)
✅ Chunk 17 finished.
🚀 Sending chunk 18/26

### finish